In [19]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What's 5 + 7?"),
    AIMessage(content="", tool_calls=[{"name": "calculator", "args": {"a": 5, "b": 7}, "id": "tool-1"}]),
    ToolMessage(content="12", tool_call_id="tool-1"),
    AIMessage(content="The answer is 12."),
    HumanMessage(content="Thanks! What is the capital of France?"),
    AIMessage(content="The capital of France is Paris.")
]


### Print Message By Type

In [22]:
for msg in messages:
    if isinstance(msg, SystemMessage):
        print(f'SYSTEM: ',msg.content)
    elif isinstance(msg, HumanMessage):
        print(f'Human: ',msg.content)
    elif isinstance(msg, AIMessage):
        if msg.tool_calls:
            last_tool =  msg.tool_calls[-1] # get the last dict from  list of dictionary
            print("AI Tool:",last_tool['name'])
            # for tool in msg.tool_calls:
            #     print(f"AI Tool: {tool['name']}")
        else:
            print(f'AI: ',msg.content)
    elif isinstance(msg, ToolMessage):
        print(f'Tools:  ',msg.content)

SYSTEM:  You are a helpful assistant.
Human:  What's 5 + 7?
AI Tool: calculator
Tools:   12
AI:  The answer is 12.
Human:  Thanks! What is the capital of France?
AI:  The capital of France is Paris.


### Group all Messages

In [23]:
conversations = []
current_block = []

for msg in messages:
    current_block.append(msg)
    if isinstance(msg, AIMessage) and not msg.tool_calls:
        conversations.append(current_block)
        current_block = []

# Print grouped conversation blocks
for i, convo in enumerate(conversations):
    print(f"\n--- Conversation {i+1} ---")
    for m in convo:
        print(f"{type(m).__name__}: {m.content}")



--- Conversation 1 ---
SystemMessage: You are a helpful assistant.
HumanMessage: What's 5 + 7?
AIMessage: 
ToolMessage: 12
AIMessage: The answer is 12.

--- Conversation 2 ---
HumanMessage: Thanks! What is the capital of France?
AIMessage: The capital of France is Paris.


### Extract only Human Message

In [24]:
user_questions = [msg.content for msg in messages if isinstance(msg, HumanMessage)]
print("Questions asked by the user:")
for q in user_questions:
    print("-", q)


Questions asked by the user:
- What's 5 + 7?
- Thanks! What is the capital of France?


### Match AI Tool Calls with Tool Results

In [26]:
tool_calls = {}
tool_results = {}

for msg in messages:
    if isinstance(msg, AIMessage) and msg.tool_calls:
        for call in msg.tool_calls:
            tool_calls[call["id"]] = call
    elif isinstance(msg, ToolMessage):
        tool_results[msg.tool_call_id] = msg.content

# Combine tool call + result
for tool_id, call in tool_calls.items():
    result = tool_results.get(tool_id)
    print(f"Tool '{call['name']}' called with args {call['args']} → result: {result}")


Tool 'calculator' called with args {'a': 5, 'b': 7} → result: 12


###  Get the Latest User Question + AI Answer

In [27]:
latest_question = None
latest_answer = None

for msg in reversed(messages):
    if not latest_answer and isinstance(msg, AIMessage) and not msg.tool_calls:
        latest_answer = msg.content
    elif not latest_question and isinstance(msg, HumanMessage):
        latest_question = msg.content
    if latest_question and latest_answer:
        break

print(f"Last Q: {latest_question}\nLast A: {latest_answer}")


Last Q: Thanks! What is the capital of France?
Last A: The capital of France is Paris.


### Get Full Conversation History as JSON-like Structure

In [29]:
conversation_log = []

for msg in messages:
    role = type(msg).__name__.replace("Message", "").lower()
    conversation_log.append({
        "role": role,
        "content": msg.content
    })

import json
print(json.dumps(conversation_log, indent=2))


[
  {
    "role": "system",
    "content": "You are a helpful assistant."
  },
  {
    "role": "human",
    "content": "What's 5 + 7?"
  },
  {
    "role": "ai",
    "content": ""
  },
  {
    "role": "tool",
    "content": "12"
  },
  {
    "role": "ai",
    "content": "The answer is 12."
  },
  {
    "role": "human",
    "content": "Thanks! What is the capital of France?"
  },
  {
    "role": "ai",
    "content": "The capital of France is Paris."
  }
]


###  Split into Conversation Rounds

In [30]:
rounds = []
temp_round = []

for msg in messages:
    temp_round.append(msg)
    if isinstance(msg, AIMessage) and not msg.tool_calls:
        rounds.append(temp_round)
        temp_round = []

# Display rounds
for i, r in enumerate(rounds):
    print(f"\n--- Round {i+1} ---")
    for m in r:
        print(f"{type(m).__name__}: {m.content}")



--- Round 1 ---
SystemMessage: You are a helpful assistant.
HumanMessage: What's 5 + 7?
AIMessage: 
ToolMessage: 12
AIMessage: The answer is 12.

--- Round 2 ---
HumanMessage: Thanks! What is the capital of France?
AIMessage: The capital of France is Paris.


### Convert Messages to Prompt Format for Re-injection

In [31]:
from langchain_core.messages import get_buffer_string

prompt = get_buffer_string(messages)
print(prompt)


System: You are a helpful assistant.
Human: What's 5 + 7?
AI: 
Tool: 12
AI: The answer is 12.
Human: Thanks! What is the capital of France?
AI: The capital of France is Paris.
